In [1]:
import os
import json
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
from PIL import Image

# Paths
image_dir = "../data/invoices-donut/valid"
json_dir = "../data/invoices-donut/donut_json/valid"

# Load filenames
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))])

# Create widgets
dropdown = widgets.Dropdown(
    options=image_files,
    description='Invoice:',
    layout=widgets.Layout(width='50%')
)

json_textarea = widgets.Textarea(
    description='JSON:',
    layout=widgets.Layout(width='80%', height='300px')
)

save_button = widgets.Button(
    description='Save Changes',
    button_style='success'
)

# Add after creating the save_button and before creating the output_widget
prev_button = widgets.Button(
    description='← Previous',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

next_button = widgets.Button(
    description='Next →',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

def go_prev(b):
    current_idx = image_files.index(dropdown.value)
    if current_idx > 0:
        dropdown.value = image_files[current_idx - 1]

def go_next(b):
    current_idx = image_files.index(dropdown.value)
    if current_idx < len(image_files) - 1:
        dropdown.value = image_files[current_idx + 1]

prev_button.on_click(go_prev)
next_button.on_click(go_next)

# Create output widget for the image display
output_widget = widgets.Output()

@output_widget.capture()
def show_data(change):
    # Clear previous output
    output_widget.clear_output()
    
    filename = change.new
    img_path = os.path.join(image_dir, filename)
    json_path = os.path.join(json_dir, os.path.splitext(filename)[0] + ".json")
    
    # Display image with smaller figure size
    with output_widget:
        img = Image.open(img_path)
        plt.figure(figsize=(8, 8))  # Reduced from (10, 10)
        plt.imshow(img)
        plt.axis('off')
        plt.title(filename)
        plt.show()
    
    # Load and show JSON
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            data = json.load(f)
        # Pretty print JSON with indentation
        json_textarea.value = json.dumps(data, indent=2)
    else:
        json_textarea.value = "JSON file not found!"
    
    # Store current file path for save function
    json_textarea.json_path = json_path

def save_json(b):
    try:
        # Parse JSON to validate
        data = json.loads(json_textarea.value)
        # Save to file
        with open(json_textarea.json_path, 'w') as f:
            json.dump(data,f, indent=2)
        print(f"✅ Saved changes to {os.path.basename(json_textarea.json_path)}")
    except json.JSONDecodeError as e:
        print(f"❌ Invalid JSON: {str(e)}")
    except Exception as e:
        print(f"❌ Error saving file: {str(e)}")

# Link dropdown to show_data function
dropdown.observe(show_data, names='value')

# Link save button
save_button.on_click(save_json)

# Update json_textarea layout
json_textarea.layout.width = '600px'  # Fixed width
json_textarea.layout.height = '600px'  # Match image height

# Create the widget layout with specific widths
nav_buttons = widgets.HBox([prev_button, next_button])
left_panel = widgets.VBox([
    dropdown,
    nav_buttons,  # Add the navigation buttons
    output_widget
], layout=widgets.Layout(width='50%'))  # Set left panel width

right_panel = widgets.VBox([
    json_textarea,
    save_button
], layout=widgets.Layout(width='50%'))  # Set right panel width

# Use HBox with spacing
widget_box = widgets.HBox([
    left_panel,
    right_panel
], layout=widgets.Layout(width='100%'))

display(widget_box)

# Update styling for better layout
display(HTML("""
<style>
.widget-textarea textarea {
    color: black !important;
    background: white !important;
    font-family: monospace;
    font-size: 14px;
    padding: 8px;
}
.jupyter-widgets-view {
    width: 100%;
}
</style>
"""))
